In [1]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from peft import (
    LoraConfig,
    get_peft_model
)
from trl import SFTTrainer, SFTConfig

# Base model and tokenizer names.
base_model_name = "google/gemma-2b-it"

# Load base model to GPU memory.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, trust_remote_code=True).to(device)

# Load tokenizer.
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Dataset for fine-tuning.
training_dataset_name = "daekeun-ml/naver-news-summarization-ko"
training_dataset = load_dataset(training_dataset_name, split="train")

# Check the data.
print(training_dataset)

# Dataset 11 is a QA sample in English.
print(training_dataset[11])
print(training_dataset[0])

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

sft_config = SFTConfig(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    # optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=50,
    learning_rate=4e-5,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    dataset_text_field="summary",
)

# View the number of trainable parameters.

peft_model = get_peft_model(base_model, peft_config)
peft_model.print_trainable_parameters()

# Initialize an SFT trainer.

sft_trainer = SFTTrainer(
    model=base_model,
    train_dataset=training_dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=sft_config,
)

# Run the trainer.
sft_trainer.train()

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 22194
})
{'date': '2022-07-04 09:12:13', 'category': 'economy', 'press': '연합뉴스 ', 'title': '특징주 삼성전자·SK하이닉스 또다시 52주 신저가 추락', 'document': '삼성전자 SK하이닉스 CG 연합뉴스TV 제공 서울 연합뉴스 김아람 기자 삼성전자와 SK하이닉스가 4일 오전 재차 52주 신저가를 경신했다. 이날 오전 9시 5분 현재 유가증권시장에서 삼성전자는 전 거래일보다 0.53% 내린 5만5천900원에 거래 중이다. SK하이닉스도 전날보다 1.14% 내린 8만6천500원에 거래됐다. 개장 직후 삼성전자는 5만5천700원 SK하이닉스는 8만6천300원까지 낙폭을 키워 이틀 연속 52주 신저가를 새로 썼다. 최근 반도체 업황 우려가 확산하면서 두 종목 주가는 하락세를 이어왔다.', 'link': 'https://n.news.naver.com/mnews/article/001/0013286226?sid=101', 'summary': '4일 오전 9시 5분 현재 유가증권시장에서 삼성전자는 전 거래일보다 0.53% 내린 5만5천900원에, SK하이닉스도 전날보다 1.14% 내린 8만6천500원에 거래되면서 삼성전자와 SK하이닉스가 4일 오전 재차 52주 신저가를 경신했다.'}
{'date': '2022-07-03 17:14:37', 'category': 'economy', 'press': 'YTN ', 'title': '추경호 중기 수출지원 총력 무역금융 40조 확대', 'document': '앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을

/home/hth021002/miniconda3/envs/test/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


RuntimeError: chunk expects at least a 1-dimensional tensor